In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import cv2

In [7]:
motion_vector_path="Fan_NOSSDAV_17\dataset\content\motion\coaster_motion.mp4"
op_path="StagedData\coaster_motion_frames"

In [26]:
def extract_frame(video_path,output_folder,fps=5,frame_width=960, frame_height=480):
    video=cv2.VideoCapture(video_path)
    original_frame_rate=int(video.get(cv2.CAP_PROP_FPS))
    original_frame_width=int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_frame_height=int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_interval=original_frame_rate//fps
    #print(frame_interval)
    #print(original_frame_rate)
    frame_count=0
    op_index=1
    while video.isOpened():
        if frame_count % frame_interval==0:
            ret,frame=video.read()
            if not ret:
                break
            frame=cv2.resize(frame,(frame_width,frame_height))
            output_file=f"{output_folder}/{op_index}.jpg"
            cv2.imwrite(output_file,frame)
            op_index+=1
        frame_count+=1
    video.release()

        

In [25]:
extract_frame(motion_vector_path,op_path)

In [ ]:
class PositionOnlyModel(nn.Module):
    def __init__(self,m_window,h_window,shared_weights=False):
        super().__init__()
        self.m_window=m_window
        self.h_window=h_window
        self.encoder_lstm=nn.LSTM(2,1024)
        self.decoder_lstm=nn.LSTM(2,1024)
        self.decoder_dense_mot=nn.Sequential(
            nn.Linear(1024,2),
            nn.Sigmoid() #sigmoid activation
        )
        self.decoder_dense_dir=nn.Sequential(
            nn.Linear(1024,2)
            nn.Tanh() # Tanh activation
        )

In [ ]:
class Encoder(nn.Module):
    def __init__(self,input_size,hidden_size):
        super().__init__()
        self.hidden_size=hidden_size
        self.lstm=nn.LSTM(input_size,hidden_size)
    
    def forward(self,input_seq):
        encoder_op, (h_n,c_n)=self.lstm(input_seq)
        return encoder_op,h_n,c_n

class Decoder(nn.Module):
    def __init__(self,output_size,hidden_size):
        super().__init__()
        self.hidden_size=hidden_size
        self.lstm=nn.LSTM()